In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import decomposition

import tensorflow as tf
import tensorflow.keras   
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications import DenseNet121, VGG16
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, accuracy_score


import pathlib
from PIL import Image

import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

In [2]:
# path for each partition of the data
data_directory = pathlib.Path('/Users/erdem/model-playground/data')

In [3]:
# creating a pathlib object for each partition
train_subdir = data_directory.joinpath("train")
validation_subdir = data_directory.joinpath("valid")
test_subdir = data_directory.joinpath("test")

In [4]:
training_images = [file for subdir in train_subdir.glob('*') for file in subdir.glob('*')]
validation_images = [file for subdir in validation_subdir.glob('*') for file in subdir.glob('*')]
test_images = [file for subdir in test_subdir.glob('*') for file in subdir.glob('*')]

print(f"Number of training examples: {len(training_images)}",
      f"\nNumber of validation examples: {len(validation_images)}",
f"\nNumber of test examples: {len(test_images)}")

Number of training examples: 5000 
Number of validation examples: 899 
Number of test examples: 902


Creating train-dev-test sets
At this step, we create a batched tf.data.dataset object from each subdirectory of our data. we determine image sizes to be (224,224), with batches of 32.

In [5]:
batch_size = 32
target_size = (224,224)

# creating the training, validation and test sets
train_set= image_dataset_from_directory(
    train_subdir,
    image_size=target_size,
    batch_size=batch_size,
)

validation_set= image_dataset_from_directory(
    validation_subdir,
    image_size=target_size,
    batch_size=batch_size,
)

test_set = image_dataset_from_directory(
    test_subdir,
    image_size=target_size,
    batch_size=batch_size,
)

Found 5000 files belonging to 5 classes.
Found 899 files belonging to 5 classes.
Found 902 files belonging to 5 classes.


In [6]:
# creating a list of labels for each set of data
validation_labels = np.concatenate([label for pic, label in validation_set], axis=0)
test_labels = np.concatenate([label for pic, label in test_set], axis=0)

# creating a pandas series from the lists to find the proportion of labels 
validation_labels_dist = pd.Series(validation_labels).value_counts(normalize = True)*100
test_labels_dist = pd.Series(test_labels).value_counts(normalize = True)*100

print("the distribution of labels in the validation set is\n",validation_labels_dist)
print("\n")
print("the distribution of labels in the test set is\n",test_labels_dist)

the distribution of labels in the validation set is
 0    48.609566
1    25.806452
2     9.010011
3     8.898776
4     7.675195
Name: proportion, dtype: float64


the distribution of labels in the test set is
 0    48.447894
1    25.831486
2     9.090909
3     8.869180
4     7.760532
Name: proportion, dtype: float64


2024-12-15 19:00:02.506091: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-12-15 19:00:02.610509: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
AUTOTUNE = tf.data.AUTOTUNE #dynamically tuning the parallelism based on available system resources

train_ds = train_set.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = validation_set.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_set.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
data_augmentation =  tf.keras.Sequential([
    layers.RandomRotation(factor=0.2),  # Random rotation (up to 20%)
    layers.RandomZoom(height_factor=0.2, width_factor=0.2),  # Random zoom
    layers.RandomFlip(mode="horizontal"),  # Random horizontal flip
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),  # Random translation
    layers.RandomContrast(factor=0.2),  # Random contrast adjustment
    layers.RandomBrightness(factor=0.2),  # Random brightness adjustment
])

In [11]:
# loading the base model
vgg16_base_model = VGG16(input_shape=(224,224,3), include_top=False, weights='/Users/erdem/model-playground/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
vgg16_base_model.trainable = False 
global_average_layer = layers.GlobalAveragePooling2D()


inputs = tf.keras.Input(shape=(224, 224, 3))
augmented = data_augmentation(inputs)
features_extracted = vgg16_base_model(augmented)
avg_pooling = global_average_layer(features_extracted)
dropout = tf.keras.layers.Dropout(0.3)(avg_pooling) #reduced the dropout rate to 0.3
outputs = layers.Dense(5, activation='softmax')(dropout)
model_vgg16 = tf.keras.Model(inputs, outputs)                                     
                                      
model_vgg16.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,717,253 (56.14 MB)

 Trainable params: 2,565 (10.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [12]:
EPOCHS = 20
# compiling the model just like before
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

# Define callbacks to improve training stability
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

model_vgg16.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=['accuracy'])


In [13]:
history_vgg16 = model_vgg16.fit(
    train_set, epochs=EPOCHS, validation_data=validation_set,
    callbacks = [early_stopping,lr_plateau])

Epoch 1/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 494s 3s/step - accuracy: 0.2023 - loss: 3.7102 - val_accuracy: 0.3415 - val_loss: 2.0205 - learning_rate: 1.0000e-04
Epoch 2/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 2586s 17s/step - accuracy: 0.2471 - loss: 2.9606 - val_accuracy: 0.4105 - val_loss: 1.6498 - learning_rate: 1.0000e-04
Epoch 3/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 501s 3s/step - accuracy: 0.2943 - loss: 2.4872 - val_accuracy: 0.4783 - val_loss: 1.4350 - learning_rate: 1.0000e-04
Epoch 4/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 479s 3s/step - accuracy: 0.3505 - loss: 2.2465 - val_accuracy: 0.5250 - val_loss: 1.3060 - learning_rate: 1.0000e-04
Epoch 5/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 511s 3s/step - accuracy: 0.3625 - loss: 2.0618 - val_accuracy: 0.5417 - val_loss: 1.2217 - learning_rate: 1.0000e-04
Epoch 6/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 490s 3s/step - accuracy: 0.4058 - loss: 1.9084 - val_accuracy: 0.5662 - val_loss: 1.1706 - learning_rate: 1.0000e-04
Epoch 7/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 475s 3s/step - a

In [14]:
vgg16_results = model_vgg16.evaluate(test_ds)

29/29 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - accuracy: 0.6594 - loss: 0.9055


In [15]:
print("VGG16 model test loss and accuracy score(in order):", vgg16_results[0],vgg16_results[1], )

VGG16 model test loss and accuracy score(in order): 0.9041709899902344 0.6596452593803406


Lets try to FINE TUNE VGG16 

Fine tuning, in most cases, is when we decide to unfreeze (some of the final and consecutive) layers of the base model(the model that we used for transfer learning) to make the model more specific to our task. As a result, we might get a higher performance score for our model.

In [16]:
# how many layers are in the base model?
print("Number of layers in the base model(VGG16): ", len(vgg16_base_model.layers))

vgg16_layers = [layer.name for layer in vgg16_base_model.layers]
print(vgg16_layers)

Number of layers in the base model(VGG16):  19
['input_layer_2', 'block1_conv1', 'block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool']


In [17]:
# swithcing on all layers to be trainable
vgg16_base_model.trainable = True

# swithcing off(freezing) all layers except the last 4 layers
fine_tune_last = 4
for layer in vgg16_base_model.layers[:-fine_tune_last]:
    layer.trainable = False

Since we are training a larger model, it might overfit so fast to the training data. So it's recommended to use a lower learning rate. instead of 1e-4, I use 1e-5 as the new learning rate. other settings are the same as before for compiling the model:

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model_vgg16.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [19]:
model_vgg16.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,717,253 (56.14 MB)

 Trainable params: 7,081,989 (27.02 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [ ]:
fine_tune_epochs = 10
total_epochs =  EPOCHS + fine_tune_epochs

history_fine = model_vgg16.fit(train_set,
                         epochs=total_epochs,
                         initial_epoch=history_vgg16.epoch[-1],
                         validation_data=validation_set)


In [ ]:
fine_tune_df = pd.DataFrame(history_fine.history)

Measuring the final results

In [ ]:
predictions = model_vgg16.predict(test_ds)
vgg16_fineTuned_results = model_vgg16.evaluate(test_ds)

In [ ]:
print("the VGG16 tuned model test loss and accuracy score(in order):", vgg16_fineTuned_results[0],
      vgg16_fineTuned_results[1] )

In [ ]:
model_vgg16.save('model_vgg16_fine_tuned.keras')